In [ ]:
import torch, gc, copy
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm
import os
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from scipy.signal import welch
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [ ]:
data = np.load('dataset.npz')

xtrs_r = data['train_s']
xtrns_r = data['train_ns']
xtes_r = data['test_s']
xtens_r = data['test_ns']

# non seizure is 0，seizure is 1
ytrs = np.zeros((xtrs_r.shape[0],1))
ytrns = np.zeros((xtrns_r.shape[0],1))
ytes = np.zeros((xtes_r.shape[0],1))
ytens = np.zeros((xtens_r.shape[0],1))

ytrs = ytrs + 1
ytrns = ytrns
ytes = ytes + 1
ytens = ytens

x_train = np.concatenate([xtrs_r,xtrns_r])
y_train = np.concatenate([ytrs,ytrns])
x_test = np.concatenate([xtes_r,xtens_r])
y_test = np.concatenate([ytes,ytens])

In [ ]:
x_train_sta = x_train
x_test_sta = x_test

x_train_sta = np.asfarray(x_train_sta)
x_test_sta = np.asfarray(x_test_sta)

x_tloader = torch.tensor(x_train_sta,dtype=torch.float32).view(-1,256,1)
y_tloader = torch.tensor(y_train, dtype=torch.long).view(y_train.shape[0])

# 将标签转换为 PyTorch 张量
y_train_tensor = torch.tensor(y_tloader)
# 计算每个类别的数量
class_counts = torch.bincount(y_train_tensor)
# 计算类别权重
class_weights = 1.0 / class_counts.float()
print(class_weights)


print(x_tloader.shape, y_tloader.shape)

x_vloader = torch.tensor(x_test_sta, dtype=torch.float32).view(-1,256,1)
y_vloader = torch.tensor(y_test, dtype=torch.long).view(y_test.shape[0])

print(x_vloader.shape, y_vloader.shape)

train_dataset = TensorDataset(x_tloader, y_tloader)
test_dataset = TensorDataset(x_vloader, y_vloader)


train_loader = DataLoader(train_dataset,batch_size=128,shuffle=True,num_workers=4)
# test_loader = DataLoader(test_dataset,batch_size=len(test_dataset),shuffle=False,num_workers=4)
test_loader = DataLoader(test_dataset,batch_size=128,shuffle=False,num_workers=4)

In [ ]:
#定义RNN架构
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(
            input_size=1,  # 每个序列点的特征维度，例如使用5个频段，则等于5
            hidden_size=64,  # 隐藏层参数
            num_layers=1,  # RNN layers
            batch_first=True,  # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
        )
        self.flag = 1
        self.out = nn.Linear(64, 2)  # 输出层

    def forward(self, x):
        r_out, (h_n, h_c) = self.rnn(x, None)  # None 表示 hidden state 会用全0的 state
        # out = self.out(r_out[:, -1, :])
        out = self.out(h_n[0])
        return out

In [ ]:
# 训练函数
def train(model, train_loader, test_loader, criterion, optimizer):
    # 训练阶段
    model.train()
    total_loss = 0
    correct = 0
    TP = 0
    FN = 0
    TN = 0
    FP = 0
    total_samples = 0
    for data, target in train_loader:
        data,target = data.cuda(),target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total_samples += target.size(0)
        correct += (predicted == target).sum().item()
        TP += ((predicted == 1) & (target == 1)).sum().item()
        FN += ((predicted == 0) & (target == 1)).sum().item()
        TN += ((predicted == 0) & (target == 0)).sum().item()
        FP += ((predicted == 1) & (target == 0)).sum().item()

    acc = correct / total_samples
    sen = TP / (TP + FN)
    spe = TN / (TN + FP)
    average_loss = total_loss / len(train_loader)
    print(f"Train Epoch {epoch+1}/{num_epochs}, Acc: {acc:.4f}, Sen: {sen:.4f}, Spe: {spe:.4f}, Loss: {loss:.4f}")
    
    # 测试阶段
    model.eval()
    total_loss = 0
    correct = 0
    TP = 0
    FN = 0
    TN = 0
    FP = 0
    total_samples = 0
    last_metric = 0

    for data, target in test_loader:
        data,target = data.cuda(),target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        total_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total_samples += target.size(0)
        correct += (predicted == target).sum().item()
        TP += ((predicted == 1) & (target == 1)).sum().item()
        FN += ((predicted == 0) & (target == 1)).sum().item()
        TN += ((predicted == 0) & (target == 0)).sum().item()
        FP += ((predicted == 1) & (target == 0)).sum().item()

    acc = correct / total_samples
    sen = TP / (TP + FN)
    spe = TN / (TN + FP)
    score = sen*spe
    average_loss = total_loss / len(test_loader)
    print(f"Test Epoch {epoch+1}/{num_epochs}, Acc: {acc:.4f}, Sen: {sen:.4f}, Spe: {spe:.4f}, Loss: {loss:.4f}")
    return model, score, sen, spe, acc

In [ ]:
 # 设置训练参数
num_epochs = 50
batch_size = 128
learning_rate = 0.0001

# 创建模型实例
lenet = RNN()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.Adam(lenet.parameters(), lr=learning_rate)

lenet.to(device)

best_model = None
b_score = 0
b_acc = 0
b_sen = 0
b_spe = 0
# 训练循环
for epoch in range(num_epochs):
    trained_model, score, sen, spe, acc = train(lenet, train_loader, test_loader, criterion, optimizer)
    if score > b_score:
        best_model = copy.deepcopy(trained_model)
        b_score = score
        b_acc = acc
        b_sen = sen
        b_spe = spe
print(f"Final metrics , Score: {b_score:.4f}, Acc: {b_acc:.4f}, Sen: {b_sen:.4f}, Spe: {b_spe:.4f}")

In [ ]:
from joblib import dump, load
# 保存model为onnx格式
# Prepare input tensor
input_tensor = torch.randn(1, 256, 1)
device = torch.device('cpu')

# Move model to the device
best_model.to(device)

directory = 'rnn'
if not os.path.exists(directory):
    os.makedirs(directory)
# Export models as ONNX files
torch.onnx.export(best_model, input_tensor, "rnn/lstm.onnx", opset_version=11)

In [ ]:
with torch.no_grad():
    outputs = best_model(x_vloader.to(device))
    ypred = torch.argmax(outputs.cpu(), dim=1)
    TP = ((ypred == 1) & (y_vloader == 1)).sum()
    FN = ((ypred == 0) & (y_vloader == 1)).sum()
    TN = ((ypred == 0) & (y_vloader == 0)).sum()
    FP = ((ypred == 1) & (y_vloader == 0)).sum()
    sen = TP / (TP + FN)
    spe = TN / (TN + FP)
    print(sen,spe)
    print(TP,FN,TN,FP)
    print(ypred.sum(),y_test.sum())

In [ ]:
# estimate model
from sklearn.metrics import classification_report,accuracy_score,roc_curve
from sklearn.metrics import confusion_matrix,auc,RocCurveDisplay,plot_confusion_matrix
import seaborn as sns

target_names = ['non-seizure', 'seizure']
print(classification_report(y_vloader, ypred, target_names=target_names))
confusion = confusion_matrix(y_vloader,ypred)
ax = sns.heatmap(confusion, annot=True, fmt='g', cmap='Blues')
ax.set_title('Confusion Matrix\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['non-seizure', 'seizure'])
ax.yaxis.set_ticklabels(['non-seizure', 'seizure'])

TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

Acc=(TP+TN)/float(TP+TN+FP+FN)
Sen=TP / float(TP+FN)
Spe=TN / float(TN+FP)
print('acc', Acc, 'sen',Sen,'spe',Spe)
plt.savefig('rnn/matrix.png', dpi=500)